## Change to project directory and wandb login

In [9]:
%cd /content/drive/MyDrive/DeepLearning_HW1

/content/drive/MyDrive/DeepLearning_HW1


In [10]:
# ! git clone https://github.com/vTheWise/MSAI437_Deep_Learning_HW1.git
%cd MSAI437_Deep_Learning_HW1/

/content/drive/MyDrive/DeepLearning_HW1/MSAI437_Deep_Learning_HW1


In [ ]:
!pip install transformers datasets evaluate
!pip install wandb

In [ ]:
import csv
import wandb
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, Features
from transformers import AutoTokenizer
from transformers import BertConfig
from transformers import AutoModelForSequenceClassification
from datasets import load_metric
import numpy as np
from transformers import TrainingArguments
from transformers import Trainer
import evaluate
import csv

In [12]:

key = '1720a18f3348a08b85d49e525888d5c8b2f73330'
wandb.login()

wandb: Currently logged in as: multitude0099 (decoding-markdowns). Use `wandb login --relogin` to force relogin


True

## Merge files

In [14]:
def preprocess_line(line):
  line = line.replace('\n', '')
  line = line.strip()

  return line

def read_file(filename, label):
  file_content = open(filename, 'r').readlines()
  # print(file_content)
  bio = []
  temp_bio = []
  i = 0
  while i < len(file_content):
    line = preprocess_line(file_content[i])
    if line == '':
      i+=1
      continue
    elif line == '<end_bio>':
      
      if label == None:
        temp_bio.append(line)
        while i <len(file_content) and line!= '[REAL]' and line!= '[FAKE]':
          line = preprocess_line(file_content[i])
          i+=1

        # temp_bio.append(line)
        temp_bio_str = ' '.join(temp_bio)
        bio.append([temp_bio_str, line])
        temp_bio = []
      else:
        temp_bio.append(line)
        temp_bio_str = ' '.join(temp_bio)
        bio.append([temp_bio_str, label])
        temp_bio = []
      # break
    else:
      temp_bio.append(line)

    i+=1
  # print(filename, len(bio))
    
  return bio

def merge_file_data(filename_list, label_list):
  data = []
  for (filename, label) in zip(filename_list, label_list):
    data = data + read_file(filename, label)
  return data
     

In [ ]:
# bio_with_labels = read_file('./mix.valid.txt', None)
# bio_with_labels[:2]
train_data = merge_file_data(['real.train.txt','fake.train.txt', 'mix.train.txt'],['[REAL]','[FAKE]', None])
val_data = merge_file_data(['real.valid.txt','fake.valid.txt', 'mix.valid.txt'],['[REAL]','[FAKE]', None])
test_data = merge_file_data(['real.test.txt','fake.test.txt', 'mix.test.txt'],['[REAL]','[FAKE]', None])

In [16]:
df_training = pd.DataFrame(train_data,columns=["bio",'label'])

df_valid = pd.DataFrame(val_data,columns=["bio",'label'])

df_test = pd.DataFrame(test_data,columns=["bio",'label'])

In [17]:
print("with duplicates:", len(df_training), len(df_valid), len(df_test))

with duplicates: 23885 2958 3156


In [18]:
df_training.drop_duplicates(inplace=True)
df_valid.drop_duplicates(inplace=True)
df_test.drop_duplicates(inplace=True)

In [45]:
print("without duplicates", len(df_training), len(df_valid), len(df_test))

without duplicates 19824 2452 2650


In [46]:
df_training.to_csv('./data/combined_train.txt', index=None, sep='\t')
df_valid.to_csv('./data/combined_valid.txt', index=None, sep='\t')
df_test.to_csv('./data/combined_test.txt', index=None, sep='\t')

## Dataloader

In [ ]:

dataset = load_dataset('csv',\
                       data_files={'train': './data/combined_train.txt', 'val': './data/combined_valid.txt', \
                                   'test': './data/combined_test.txt'}, delimiter = '\t')
# dataset.Features = Features('bio', 'label')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [22]:
tokenizer("this is a fake bio")

{'input_ids': [101, 1142, 1110, 170, 8406, 25128, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [23]:
def tokenize_data(example):
    return tokenizer(example['bio'], padding='max_length', truncation = True)

dataset = dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/19824 [00:00<?, ? examples/s]

Map:   0%|          | 0/2452 [00:00<?, ? examples/s]

Map:   0%|          | 0/2650 [00:00<?, ? examples/s]

In [24]:
def transform_labels(example):
  label = example['label']
  if label == '[REAL]':
    num = 0
  
  else:
    num = 1

  return {'labels': num}
dataset = dataset.map(transform_labels, remove_columns = ['bio', 'label'])

Map:   0%|          | 0/19824 [00:00<?, ? examples/s]

Map:   0%|          | 0/2452 [00:00<?, ? examples/s]

Map:   0%|          | 0/2650 [00:00<?, ? examples/s]

In [25]:
for i in dataset['train']:
  print(i)
  break

{'input_ids': [101, 133, 1838, 168, 25128, 135, 134, 12091, 15095, 139, 2149, 1732, 134, 12091, 15095, 139, 2149, 1732, 117, 143, 8900, 113, 1367, 1318, 5064, 782, 1627, 1351, 2278, 114, 1108, 170, 1418, 14646, 1105, 3806, 119, 1124, 1108, 1255, 1107, 7383, 119, 1124, 3024, 1121, 3414, 1179, 2138, 1105, 140, 3814, 1361, 1531, 117, 3900, 1107, 4123, 1105, 1310, 1844, 1120, 1103, 5458, 118, 20067, 1419, 1107, 157, 26664, 1670, 117, 4280, 119, 1247, 1119, 1872, 112, 170, 15418, 9515, 112, 23638, 1134, 8824, 1344, 191, 7409, 6718, 1106, 1129, 3890, 117, 1543, 1936, 1103, 20911, 1104, 20771, 15700, 119, 1130, 3698, 1119, 1108, 1189, 6229, 1104, 9126, 7376, 1120, 4849, 1531, 1498, 1105, 1107, 3588, 1427, 1106, 1239, 1104, 6490, 1112, 170, 1844, 6229, 1107, 1103, 1951, 1104, 8937, 119, 1247, 1119, 7902, 1113, 1103, 1718, 1104, 17599, 11428, 5005, 1105, 21359, 2897, 20739, 119, 1124, 1281, 1103, 2889, 12786, 13976, 3991, 3803, 1105, 3449, 1105, 1103, 3183, 155, 1818, 2821, 3803, 117, 1103, 298

## Training

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",  num_labels=2)

In [27]:
train_dataset = dataset['train'].shuffle(seed=2020)
eval_dataset = dataset['val'].shuffle(seed=2020)
test_dataset = dataset['test'].shuffle(seed=2020)

In [ ]:

acc_metric = load_metric('accuracy')

In [29]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return acc_metric.compute(predictions=predictions, references=labels)

In [30]:


training_args = TrainingArguments(output_dir = 'bert_base_finetuned_fake_bio_detector', per_gpu_train_batch_size = 16,\
                                  per_device_eval_batch_size=64, \
                                  do_eval = True, \
                                  num_train_epochs = 3, \
                                  report_to = 'wandb', \
                                  logging_strategy="steps", \
                                  evaluation_strategy="steps", \
                                  logging_steps = 300,\
                                  save_steps = 300,\
                                  load_best_model_at_end=True,\
                                  metric_for_best_model='accuracy')
                                  # push_to_hub=True)

In [31]:


trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset,\
    compute_metrics=compute_metrics
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:
%%wandb
trainer.train()

## Model Evaluation on testset

In [54]:
preds = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 2650
  Batch size = 64


In [59]:
print("test accuracy", preds.metrics['test_accuracy'])

test accuracy 0.9237735849056604


## Load and run best model on Blind set

In [33]:
dataset = load_dataset('csv',\
                       data_files={'blind_test': './data/processed_blind.test.txt'}, delimiter = '\n')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating blind_test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-35c6bb8bcbf3a9fb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
def tokenize_blind_data(example):
    return tokenizer(example['bio'], padding='max_length', truncation = True)

dataset = dataset.map(tokenize_blind_data, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:

test_args = TrainingArguments(
    output_dir = './bert_base_finetuned_fake_bio_detector/checkpoint-1800',
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 64,   
    dataloader_drop_last = False    
)

from datasets import load_metric
acc_metric = load_metric('accuracy')

import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return acc_metric.compute(predictions=predictions, references=labels)

# init trainer
trainer = Trainer(
              model = model, 
              args = test_args, 
              compute_metrics = compute_metrics)



In [39]:
test_results = trainer.predict(dataset['blind_test'])
test_labels = test_results.predictions.argmax(-1)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: bio. If bio are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 500
  Batch size = 64


In [50]:
test_labels.shape

(500,)

## Write labels to CSV

In [61]:

with open('./blind_test_results.csv', 'w') as f:
  writer = csv.DictWriter(f, fieldnames = {'label'})
  writer.writeheader()
  for l in test_labels:
    if l == 1:
      writer.writerow({'label' : '[FAKE]'})
    else:
      writer.writerow({'label' : '[REAL]'})
    # writer.writerow({'Organization' : 'Microsoft',
    #                   'Established': '1975',
    #                   'CEO': 'Satya Nadella'})
    # writer.writerow({'Organization' : 'Nokia',
    #                   'Established': '1865',
    #                   'CEO': 'Rajeev Suri'})